# Labs 5

## 2) Łączenie (sekwencje) ‘akcji’ I/O — operatory >> (`then`) i >>= (`bind`), notacja do

**1. Napisać odpowiedniki `echo3` i `dialog` wykorzystujące notację `do`.**

**echo3 z użyciem do**

In [1]:
echo3 :: IO()
echo3 = getLine >>= \l1 -> getLine >>= \l2 -> putStrLn $ l1 ++ l2

**echo3 bez użycia do**

In [2]:
echo3' :: IO()
echo3' = do
    l1 <- getLine
    l2 <- getLine
    putStrLn $ l1 ++ l2

**dialog z użyciem do**

In [3]:
dialog :: IO()
dialog = putStr "What is your happy number? "
    >> getLine
    >>= \choice -> let num = read choice :: Int in
        if num == 7
        then putStrLn "Ah, lucky 7!"
        else if odd num
            then putStrLn "Odd number! That's most people's choice..."
            else putStrLn "Hm, even number? Unusual!"

**dialog bez użycia do**

In [4]:
dialog' :: IO()
dialog' = do
    putStr "What is your happy number? "
    choice <- getLine
    let num = read choice :: Int in
        if num == 7
        then putStrLn "Ah, lucky 7!"
        else if odd num
            then putStrLn "Odd number! That's most people's choice..."
            else putStrLn "Hm, even number? Unusual!"

**2. Napisać odpowiednik `twoQuestions` bez użycia notacji `do`.**

**twoQuestions z użyciem do**

In [5]:
twoQuestions :: IO ()
twoQuestions = do
  putStr "What is your name? "
  name <- getLine
  putStr "How old are you? "
  age <- getLine
  print (name,age)

**twoQuestions bez użycia do**

In [6]:
twoQuestions :: IO ()
twoQuestions = putStr "What is your name? "
    >> getLine
    >>= \name -> putStr "How old are you? "
    >> getLine
    >>= \age -> print(name, age)

**3. Napisać ‘akcję’ `getLine'` odpowiadającą `getLine` z biblioteki `Prelude`.**

In [7]:
getLine' :: IO String
getLine' = do
    char <- getChar
    if char == '\n'
    then return ""
    else do
        line <- getLine
        return (char:line)

## 6) Funktory 2: dołączanie typów użytkownika do klasy `Functor`

**4. Sprawdzić możliwość automatycznego wygenerowania instancji `Functor` dla typu `MyList` (klauzula deriving)**

In [8]:
{-# LANGUAGE DeriveFunctor #-}
data MyList a = EmptyList
              | Cons a (MyList a) deriving (Show, Functor)

In [9]:
list = Cons 5 (Cons 3 (Cons 29 EmptyList))
fmap (+5) list

Cons 10 (Cons 8 (Cons 34 EmptyList))

**5. Napisać własną implementację funktora (`instance Functor`), a następnie sprawdzić możliwość jej automatycznego wygenerowania dla drzewa binarnego zdefiniowanego jako**

**data BinTree a = EmptyBT | NodeBT a (BinTree a) (BinTree a) deriving (Show)**

In [10]:
data BinTree a = EmptyBT
             | NodeBT a (BinTree a) (BinTree a) deriving (Show)

In [11]:
instance Functor BinTree where
    fmap _ EmptyBT = EmptyBT
    fmap f (NodeBT a lt rt) = NodeBT (f a) (fmap f lt) (fmap f rt)

In [12]:
data BinTree' a = EmptyBT'
             | NodeBT' a (BinTree' a) (BinTree' a) deriving (Show, Functor)

In [13]:
binTree' = NodeBT' 5 (NodeBT' 4 (NodeBT' 7 EmptyBT' EmptyBT') EmptyBT') EmptyBT'
fmap (*4) binTree'

NodeBT' 20 (NodeBT' 16 (NodeBT' 28 EmptyBT' EmptyBT') EmptyBT') EmptyBT'

**6. Napisać implementacje funktora (`instance Functor`) dla następujących typów:**

**a) newtype Pair b a = Pair { getPair :: (a,b) } -- fmap should change the first element**

In [14]:
newtype Pair b a = Pair { getPair :: (a,b) } deriving Show

In [15]:
instance Functor (Pair b) where
    fmap f Pair {getPair = (a, b)} = Pair {getPair = (f a, b)}

In [16]:
fmap (*4) (Pair (6, 3))

Pair {getPair = (24,3)}

**b) data Tree2 a = EmptyT2 | Leaf a | Node (Tree2 a) a (Tree2 a) deriving Show**

In [17]:
data Tree2 a = EmptyT2
             | Leaf a
             | Node (Tree2 a) a (Tree2 a) deriving Show

In [18]:
instance Functor Tree2 where
    fmap _ EmptyT2 = EmptyT2
    fmap f (Leaf a) = Leaf (f a)
    fmap f (Node lt a rt) = Node (fmap f lt) (f a) (fmap f rt)

In [19]:
tree2 = Node (Leaf 3) 5 (Leaf 9)

In [20]:
fmap (+5) tree2

Node (Leaf 8) 10 (Leaf 14)

**c) data GTree a = Leaf a | GNode [GTree a] deriving Show**

In [21]:
data GTree a = Leaf a
             | GNode [GTree a] deriving Show

In [22]:
instance Functor GTree where
    fmap f (Leaf a) = Leaf (f a)
    fmap f (GNode []) = GNode []
    fmap f (GNode [x]) = GNode [fmap f x]

In [23]:
gTree = GNode [GNode [Leaf 7]]

In [24]:
fmap (+4) gTree

GNode [GNode [Leaf 11]]

## 8) Funktory aplikatywne 2: dołączanie typów użytkownika do klasy `Applicative`

**7. Napisać implementacje funktora aplikatywnego (`instance Applicative`) dla typu:**

**newtype MyTriple a = MyTriple (a,a,a) deriving Show**

In [25]:
newtype MyTriple a = MyTriple (a,a,a) deriving Show

In [26]:
instance Functor MyTriple where
    fmap f (MyTriple (a, b, c)) = MyTriple (f a, f b, f c)

In [27]:
instance Applicative MyTriple where
    pure a = MyTriple (a,a,a)
    (MyTriple (f, g, h)) <*> (MyTriple (a, b, c)) = MyTriple (f a, g b, h c)

In [28]:
fmap (+7) (MyTriple (6, 1, 0))

MyTriple (13,8,7)